In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [6]:
z=torch.FloatTensor([5,2,1]) 
y=F.softmax(z, dim=0)
print(y)
print(y.sum())

tensor([0.9362, 0.0466, 0.0171])
tensor(1.)


In [21]:
z=torch.rand(3,5, requires_grad=True)
h=F.softmax(z, dim=1) #세로 dim=0, 가로 dim=1
print(h)
print(h.sum(dim=1))

tensor([[0.1849, 0.1710, 0.2070, 0.2455, 0.1916],
        [0.2126, 0.2067, 0.1581, 0.2146, 0.2081],
        [0.2197, 0.2268, 0.1367, 0.2639, 0.1528]], grad_fn=<SoftmaxBackward0>)
tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


In [26]:
y=torch.randint(5,(3,)).long()
print(y)
y_one_hot=torch.zeros_like(h)
print(y_one_hot)
y_one_hot.scatter_(1, y.unsqueeze(1),1)
print(y_one_hot)

tensor([4, 4, 0])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.]])


In [27]:
print(y.unsqueeze(1))

tensor([[4],
        [4],
        [0]])


In [35]:
cost=(y_one_hot*-torch.log(h)).sum(dim=1).mean()
print(cost)
print(z)

tensor(1.5792, grad_fn=<MeanBackward0>)
tensor([[0.1581, 0.0801, 0.2709, 0.4418, 0.1935],
        [0.6829, 0.6547, 0.3868, 0.6922, 0.6616],
        [0.8053, 0.8367, 0.3307, 0.9885, 0.4422]], requires_grad=True)


In [29]:
torch.log(F.softmax(z, dim=1)) # 아래와 동일

tensor([[-1.6880, -1.7659, -1.5752, -1.4043, -1.6525],
        [-1.5484, -1.5766, -1.8446, -1.5392, -1.5698],
        [-1.5153, -1.4838, -1.9899, -1.3320, -1.8784]], grad_fn=<LogBackward0>)

In [30]:
F.log_softmax(z, dim=1) # 위와 동일

tensor([[-1.6880, -1.7659, -1.5752, -1.4043, -1.6525],
        [-1.5484, -1.5766, -1.8446, -1.5392, -1.5698],
        [-1.5153, -1.4838, -1.9899, -1.3320, -1.8784]],
       grad_fn=<LogSoftmaxBackward0>)

In [31]:
(y_one_hot*-torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.5792, grad_fn=<MeanBackward0>)

In [32]:
(y_one_hot*-F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.5792, grad_fn=<MeanBackward0>)

In [33]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.5792, grad_fn=<NllLossBackward0>)

In [34]:
F.cross_entropy(z, y) # 이것 포함 위의 3개 동일 

tensor(1.5792, grad_fn=<NllLossBackward0>)

In [36]:

import torch.nn as nn
import torch.optim as optim

In [40]:
torch.manual_seed(1)
x_train=[[1,2,1,3],
         [2,1,3,2],
         [3,1,3,4],
         [4,1,5,5],
         [1,7,5,5],
         [1,2,5,6],
         [1,6,6,6],
         [1,7,7,7]]
y_train= [2,2,2,1,1,1,0,0]
x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [52]:
# y_one_hot=torch.zeros_like(x_train)
y_one_hot=torch.zeros(8,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
print(y,y_one_hot)

tensor([4, 4, 0]) tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [53]:
# low_level
W=torch.zeros((4,3), requires_grad=True)
b=torch.zeros((1,3), requires_grad=True)

print(W,b)

optimizer=optim.SGD([W,b], lr=0.1)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True) tensor([[0., 0., 0.]], requires_grad=True)


In [54]:
n_epochs=1000
for epoch in range(n_epochs):
    y_hat=F.softmax(x_train.matmul(W)+b, dim=1)
    cost=(y_one_hot*-torch.log(y_hat)).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print('epoch=', epoch, 'cost=', cost.item())

epoch= 0 cost= 1.0986123085021973
epoch= 100 cost= 0.7300646305084229
epoch= 200 cost= 0.637054443359375
epoch= 300 cost= 0.5714213848114014
epoch= 400 cost= 0.5157002210617065
epoch= 500 cost= 0.46481865644454956
epoch= 600 cost= 0.4165162444114685
epoch= 700 cost= 0.36960649490356445
epoch= 800 cost= 0.32354995608329773
epoch= 900 cost= 0.278940886259079


In [56]:
# high_level 구현
W=torch.zeros((4,3), requires_grad=True)
b=torch.zeros((1,3), requires_grad=True)

print(W,b)

optimizer=optim.SGD([W,b], lr=0.1)

for epoch in range(n_epochs):
    z=x_train.matmul(W)+b
    cost=F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print('epoch=', epoch, 'cost=', cost.item())

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True) tensor([[0., 0., 0.]], requires_grad=True)
epoch= 0 cost= 1.0986123085021973
epoch= 100 cost= 0.7300646305084229
epoch= 200 cost= 0.637054443359375
epoch= 300 cost= 0.5714214444160461
epoch= 400 cost= 0.5157002806663513
epoch= 500 cost= 0.4648188650608063
epoch= 600 cost= 0.41651639342308044
epoch= 700 cost= 0.369606614112854
epoch= 800 cost= 0.32354989647865295
epoch= 900 cost= 0.2789408564567566


In [57]:
model=nn.Linear(4,3)
optimizer=optim.SGD(model.parameters(), lr=0.01)

for epoch in range(n_epochs):
    y_hat=model(x_train)
    cost=F.cross_entropy(y_hat, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print('epoch=', epoch, 'cost=', cost.item())

epoch= 0 cost= 1.6563588380813599
epoch= 100 cost= 0.9077876210212708
epoch= 200 cost= 0.8007525205612183
epoch= 300 cost= 0.7450324892997742
epoch= 400 cost= 0.7041950225830078
epoch= 500 cost= 0.671798586845398
epoch= 600 cost= 0.6452271342277527
epoch= 700 cost= 0.622939944267273
epoch= 800 cost= 0.6038928627967834
epoch= 900 cost= 0.5873399972915649


In [58]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3) # Output이 3!

    def forward(self, x):
        return self.linear(x)    

In [59]:
model=SoftmaxClassifierModel()
optimizer=optim.SGD(model.parameters(), lr=0.01)

for epoch in range(n_epochs):
    y_hat=model(x_train)
    cost=F.cross_entropy(y_hat, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100==0:
        print('epoch=', epoch, 'cost=', cost.item())
    

epoch= 0 cost= 2.6547322273254395
epoch= 100 cost= 0.8455586433410645
epoch= 200 cost= 0.7601957321166992
epoch= 300 cost= 0.7084351778030396
epoch= 400 cost= 0.6710707545280457
epoch= 500 cost= 0.6419177055358887
epoch= 600 cost= 0.6181488633155823
epoch= 700 cost= 0.5981897711753845
epoch= 800 cost= 0.5810540914535522
epoch= 900 cost= 0.5660780072212219
